In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
# Define dataset paths
data_dir = "dataset_fer"
train_dir = f"{data_dir}/train"
test_dir = f"{data_dir}/test"

In [3]:
# Image parameters
IMG_SIZE = (48, 48)  # FER-2013 images are 48x48 grayscale
BATCH_SIZE = 32

In [4]:
# Data Generators (without augmentation for now)
train_datagen = ImageDataGenerator(rescale=1.0 / 255)
test_datagen = ImageDataGenerator(rescale=1.0 / 255)

In [5]:
# Load datasets
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    color_mode="grayscale",  # Ensure grayscale
    batch_size=BATCH_SIZE,
    class_mode="categorical",
    shuffle=True,
)

Found 28709 images belonging to 7 classes.


In [6]:
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=IMG_SIZE,
    color_mode="grayscale",
    batch_size=BATCH_SIZE,
    class_mode="categorical",
    shuffle=False,
)

Found 7178 images belonging to 7 classes.


In [7]:
# Check class labels
class_labels = list(train_generator.class_indices.keys())
print("Class Labels:", class_labels)

Class Labels: ['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']


### Define the Model Architecture

📜 Steps in Model Creation

1️⃣ Build a ResNet-50 Model (without pretrained weights)

2️⃣ Modify the final layer for 7 emotion classes

3️⃣ Compile with an appropriate optimizer & loss function

4️⃣ Train & evaluate the model

In [8]:
import tensorflow as tf
from tensorflow.keras import layers, models

In [9]:
# Define input shape
INPUT_SHAPE = (48, 48, 1)  # FER-2013 images are grayscale
NUM_CLASSES = 7  # 7 emotion classes

In [10]:
# ResNet-50 from scratch
def build_resnet50():
    base_model = tf.keras.applications.ResNet50(
        include_top=False,
        weights=None,  # Training from scratch
        input_shape=INPUT_SHAPE,
    )

    # Add custom classifier
    x = layers.GlobalAveragePooling2D()(base_model.output)
    x = layers.Dense(512, activation="relu")(x)
    x = layers.Dropout(0.5)(x)
    output = layers.Dense(NUM_CLASSES, activation="softmax")(x)

    model = models.Model(inputs=base_model.input, outputs=output)
    return model

In [11]:
# Compile model
model = build_resnet50()
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss="categorical_crossentropy",
    metrics=["accuracy"],
)

In [12]:
# Print model summary
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 48, 48, 1)]          0         []                            
                                                                                                  
 conv1_pad (ZeroPadding2D)   (None, 54, 54, 1)            0         ['input_1[0][0]']             
                                                                                                  
 conv1_conv (Conv2D)         (None, 24, 24, 64)           3200      ['conv1_pad[0][0]']           
                                                                                                  
 conv1_bn (BatchNormalizati  (None, 24, 24, 64)           256       ['conv1_conv[0][0]']          
 on)                                                                                          

In [13]:
# Callbacks
from tensorflow.keras import callbacks
checkpoint_cb = callbacks.ModelCheckpoint(
    "best_model.h5", save_best_only=True, monitor="val_accuracy", mode="max"
)
early_stopping_cb = callbacks.EarlyStopping(patience=5, restore_best_weights=True)


In [ ]:
# Train model
EPOCHS = 30  # Adjust as needed
history = model.fit(
    train_generator,
    epochs=EPOCHS,
    validation_data=test_generator,
    callbacks=[checkpoint_cb, early_stopping_cb],
)

Epoch 1/30
 21/898 [..............................] - ETA: 16:58 - loss: 5.0378 - accuracy: 0.1935